### Run Dataflow jobs

In [8]:
%run PeopleOver65_dataflow.py

In [1]:
%run PeopleOver85_dataflow.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1971: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(
/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:900: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location


### Verify Primary Key Constraints for Transformed Tables

In [1]:
%%bigquery
SELECT COUNT(GEOID) GEOID_distinct_count FROM
  (SELECT DISTINCT GEOID
  FROM US_Census_Bureau_refined.PeopleOver65_Dataflow);

GEOID_distinct_count
0                  7283

In [2]:
%%bigquery
SELECT COUNT(GEOID) GEOID_count FROM
  (SELECT GEOID
  FROM US_Census_Bureau_refined.PeopleOver65_Dataflow);

GEOID_count
0         7283

In [4]:
%%bigquery
SELECT COUNT(GEOID) GEOID_distinct_count FROM
  (SELECT DISTINCT GEOID
  FROM US_Census_Bureau_refined.PeopleOver85_Dataflow);

GEOID_distinct_count
0                  7283

In [5]:
%%bigquery
SELECT COUNT(GEOID) GEOID_count FROM
  (SELECT GEOID
  FROM US_Census_Bureau_refined.PeopleOver85_Dataflow);

GEOID_count
0         7283

In [1]:
# table transformed by beam has no foreign keys, so no fk constraint verification required

In [5]:
dataset_id = "insights"

In [6]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'lucid-hook-288402:insights' successfully created.


### Cross dataset queries

In [8]:
# create view that shows combined_key, estimate, geoname, and deaths on 11/3/20
# based on state join between PeopleOver65 table and COVID19_USdeaths

In [46]:
%%bigquery

CREATE OR REPLACE VIEW insights.deaths_and_estimate_65 AS
SELECT Combined_Key, ESTIMATE, GEONAME, _11_3_20
FROM
(SELECT Combined_Key, ESTIMATE, GEONAME, _11_3_20
 FROM US_Census_Bureau_refined.PeopleOver65_Dataflow
 JOIN JHU_Daily_Reports_refined.COVID19_USdeaths
 ON PeopleOver65_Dataflow.STATE = COVID19_USdeaths.Province_State);

Empty DataFrame
Columns: []
Index: []

In [9]:
# create view that shows combined_key, estimate, geoname, and deaths on 11/3/20
# based on state join between PeopleOver85 table and COVID19_USdeaths

In [1]:
%%bigquery

CREATE OR REPLACE VIEW insights.deaths_and_estimate_85 AS
SELECT Combined_Key, ESTIMATE, GEONAME, _11_3_20
FROM
(SELECT Combined_Key, ESTIMATE, GEONAME, _11_3_20
 FROM US_Census_Bureau_refined.PeopleOver85_Dataflow
 JOIN JHU_Daily_Reports_refined.COVID19_USdeaths
 ON PeopleOver85_Dataflow.STATE = COVID19_USdeaths.Province_State);

Empty DataFrame
Columns: []
Index: []

In [13]:
# create view that shows combined_key, geoname and estimate, deaths on 11/1/20,
# and the population of [combined_key] location.
# based on state join between PeopleOver65 table and COVID19_USdeaths as well
# as a foregin key join between PeopleOver65 and COVID19_USPopulations

In [11]:
%%bigquery

CREATE OR REPLACE VIEW insights.deaths_estimate_pop_65 AS
SELECT Combined_Key, _11_1_20, ESTIMATE, GEONAME, Population
 FROM US_Census_Bureau_refined.PeopleOver65_Dataflow
 JOIN JHU_Daily_Reports_refined.COVID19_USdeaths
 ON PeopleOver65_Dataflow.STATE = COVID19_USdeaths.Province_State
 JOIN JHU_Daily_Reports_refined.COVID19_USPopulations
 ON COVID19_USdeaths.UID = COVID19_USPopulations.UID

Empty DataFrame
Columns: []
Index: []